<h1 align=center><font size = 5>Clustering Neighborhoods in Toronto</font></h1>

This assignment is to explore and cluster neighborhood in Toronto.

## Import all essential pacakges
1. [Pandas](https://pandas.pydata.org/pandas-docs/stable/)
2. [Numpy](https://docs.scipy.org/doc/)
3. [Folium](https://python-visualization.github.io/folium/)
4. [Requests](https://2.python-requests.org/en/master/)
5. [Scikit Learn](https://scikit-learn.org/stable/documentation.html)

Note: when using pandas to import html, it requires lxml to be installed, this could be done by run:
    ```pip install lxml```

If you want to set this behaviour for all instances of Jupyter (Notebook and Console), simply create a file ~/.ipython/profile_default/ipython_config.py with the lines below.

```python
c = get_config()
# Run all nodes interactively
c.InteractiveShell.ast_node_interactivity = "all"
```

In [50]:
import pandas as pd
import numpy as np
import folium
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

## Assignment requirement
1. Use the Notebook to build the code to scrape the following [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)
2. The imported data frame should look like this:
<img src="img/example_df.png" style="height:500px">
3. To create the above dataframe:

    - The dataframe will consist of three columns: ___PostalCode___, ___Borough___, and ___Neighborhood___
    - Only process the cells that have an assigned borough. ___Ignore cells with a borough that is Not assigned.___
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row ___with the neighborhoods separated with a comma___ as shown in row 11 in the above table.

    * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Prepare Neighbourhood data

### Import Toronto's neighbourhood info from this [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [6]:
with open("toronto_nb.html") as f:
    tables = pd.read_html(f.read())

toronto_nb = tables[0]
toronto_nb = toronto_nb[toronto_nb.Borough != 'Not assigned']
indeces = toronto_nb[(toronto_nb.Neighbourhood == 'Not assigned')].index
for index in indeces:
    toronto_nb.loc[index, 'Neighbourhood'] = toronto_nb.loc[index, 'Borough']
table = pd.pivot_table(toronto_nb, values=['Neighbourhood'], columns=['Postcode', 'Borough'], aggfunc=lambda x: ','.join(x))
toronto_nb = table.reset_index()
toronto_nb.drop('level_0', axis=1, inplace=True)
toronto_nb.rename(columns = {0: 'Neighbourhood'}, inplace=True)
toronto_nb.head()
toronto_nb.tail()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
102,M9W,Etobicoke,Northwest


##### Tips:
The wiki page contains 3 tables, the first one is the one we wanted, so get it directly fromt the list[0]
The match argument is worth to try, but seems like it requires other additional pacakges.

### Get Geo coordinate of each neighbourhood
Get latitude and longitude using geocoder package or import geo info directly from the csv file to create the following dataframe:
<img src="img/toronto_geo.png" style="height:500px">

In [7]:
toronto_geo = pd.read_csv('Geospatial_Coordinates.csv')
toronto_geo.head()
toronto_nb = toronto_nb.set_index('Postcode').join(toronto_geo.set_index('Postal Code'), how='left')
#toronto_nb = pd.merge(toronto_nb, toronto_geo, how='left', on=['Postcode'])
toronto_nb.reset_index(inplace=True)
toronto_nb.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Create a map of Toronto with neighborhoods superimposed on top.

In [8]:
latitude = 43.6529
longitude = -79.3849
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_nb['Latitude'], toronto_nb['Longitude'], toronto_nb['Borough'], toronto_nb['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [73]:
CLIENT_ID = 'your-foursquare-id' # your Foursquare ID
CLIENT_SECRET = 'your-foursquare-secret' # your Foursquare Secret
VERSION = '20190719' # Foursquare API version
LIMIT = 10 # Number of results to return, up to 50

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-foursquare-id
CLIENT_SECRET:your-foursquare-secret


In [14]:
toronto_nb.loc[0, 'Neighbourhood']
neighborhood_latitude = toronto_nb.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_nb.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_nb.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

'Rouge,Malvern'

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [17]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            neighborhood_latitude, 
            neighborhood_longitude, 
            500, 
            LIMIT)
results = requests.get(url).json()

{'meta': {'code': 200, 'requestId': '5d315ab59ba3e5002c517843'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

#### Explore Neighborhoods in Toronto
This function repeat the same process to explore all the venues in a list of neighborhoods.

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        response = requests.get(url).json()
        if(response['meta']['code'] != 200):
            print('Explore venues for neighborhood {} failed with code: {}'.format(name, response['meta']['code']))
            continue
        else:
            results = response['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
toronto_venues = getNearbyVenues(names=toronto_nb['Neighbourhood'],
                                 latitudes=toronto_nb['Latitude'],
                                 longitudes=toronto_nb['Longitude'])
toronto_venues.shape
toronto_venues.head()
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

(695, 7)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",10,10,10,10,10,10
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10


There are 176 uniques categories.


## Analyze Each Neighborhood

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# And let's examine the new dataframe
toronto_onehot.shape
toronto_onehot.head()

(695, 176)

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape
toronto_grouped

(99, 176)

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.1,0.0,0.0,0.0,0.0,0.000000
1,Agincourt,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
4,"Alderwood,Long Branch",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
6,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000
8,Berczy Park,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.1,0.0,0.0,0.0,0.0,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000


#### Print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                venue  freq
0           Speakeasy   0.1
1          Food Court   0.1
2        Concert Hall   0.1
3    Greek Restaurant   0.1
4  Seafood Restaurant   0.1


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2  Clothing Store  0.25
3  Sandwich Place  0.25
4     Yoga Studio  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
            venue  freq
0      Playground   0.5
1            Park   0.5
2     Yoga Studio   0.0
3           Motel   0.0
4  Medical Center   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
            venue  freq
0   Grocery Store  0.22
1     Pizza Place  0.11
2     Coffee Shop  0.11
3      Beer Store  0.11
4  Sandwich Place  0.11


----Alderwood,Long Branch----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2                Pool   0.1
3         Coffee Shop   0.1


                        venue  freq
0  Construction & Landscaping  0.25
1            Basketball Court  0.25
2                        Park  0.25
3                      Bakery  0.25
4                       Motel  0.00


----East Birchmount Park,Ionview,Kennedy Park----
                venue  freq
0    Department Store  0.17
1          Hobby Shop  0.17
2         Coffee Shop  0.17
3  Chinese Restaurant  0.17
4      Discount Store  0.17


----East Toronto----
               venue  freq
0               Park   0.4
1        Coffee Shop   0.2
2  Convenience Store   0.2
3        Pizza Place   0.2
4        Yoga Studio   0.0


----Emery,Humberlea----
                      venue  freq
0            Baseball Field   1.0
1               Yoga Studio   0.0
2                    Market   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----Fairview,Henry Farm,Oriole----
                 venue  freq
0             Pharmacy   0.1
1               Bakery   0.1
2         Burger Joint   

                     venue  freq
0              Cheese Shop   0.1
1                Bookstore   0.1
2  Comfort Food Restaurant   0.1
3           Ice Cream Shop   0.1
4              Fish Market   0.1


----The Annex,North Midtown,Yorkville----
                 venue  freq
0                 Café   0.2
1          Pizza Place   0.1
2  American Restaurant   0.1
3         Burger Joint   0.1
4    Indian Restaurant   0.1


----The Beaches----
               venue  freq
0  Health Food Store  0.25
1              Trail  0.25
2                Pub  0.25
3        Yoga Studio  0.00
4              Motel  0.00


----The Beaches West,India Bazaar----
               venue  freq
0                Gym   0.1
1     Ice Cream Shop   0.1
2  Fish & Chips Shop   0.1
3       Liquor Store   0.1
4       Burger Joint   0.1


----The Danforth West,Riverdale----
              venue  freq
0  Greek Restaurant   0.3
1    Ice Cream Shop   0.2
2       Yoga Studio   0.1
3           Brewery   0.1
4    Cosmetics Shop   0.1


--

#### Create a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Speakeasy,Plaza,Hotel,Seafood Restaurant,Food Court,Asian Restaurant,Steakhouse,Vegetarian / Vegan Restaurant,Concert Hall,Greek Restaurant
1,Agincourt,Breakfast Spot,Clothing Store,Sandwich Place,Lounge,Women's Store,Department Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Women's Store,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Pizza Place,Sandwich Place,Coffee Shop,American Restaurant,Comic Shop
4,"Alderwood,Long Branch",Pizza Place,Gym,Athletics & Sports,Coffee Shop,Pool,Pub,Sandwich Place,Skating Rink,Pharmacy,Arts & Crafts Store


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 3, 0, 3, 0, 3, 0, 0], dtype=int32)

In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_nb.rename(columns={'Neighbourhood': 'Neighborhood'})

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged[~np.isnan(toronto_merged['Cluster Labels'])]

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Print Shop,Women's Store,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4.0,Bar,Women's Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Spa,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Intersection,Mexican Restaurant,Discount Store,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean Restaurant,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Bank,Bakery,Women's Store,Dessert Shop,Electronics Store


In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Spa,Rental Car Location,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Intersection,Mexican Restaurant,Discount Store,Diner
4,Scarborough,0.0,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Bank,Bakery,Women's Store,Dessert Shop,Electronics Store
7,Scarborough,0.0,Bus Line,Bakery,Bus Station,Fast Food Restaurant,Metro Station,Soccer Field,Park,Cuban Restaurant,Construction & Landscaping,Eastern European Restaurant
8,Scarborough,0.0,American Restaurant,Motel,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
9,Scarborough,0.0,College Stadium,Skating Rink,General Entertainment,Café,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
10,Scarborough,0.0,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Thrift / Vintage Store,Light Rail Station,Vietnamese Restaurant,Construction & Landscaping,Department Store,Comfort Food Restaurant
11,Scarborough,0.0,Middle Eastern Restaurant,Auto Garage,Shopping Mall,Sandwich Place,Breakfast Spot,Accessories Store,Convenience Store,Dessert Shop,Eastern European Restaurant,Concert Hall
12,Scarborough,0.0,Breakfast Spot,Clothing Store,Sandwich Place,Lounge,Women's Store,Department Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
13,Scarborough,0.0,Pizza Place,Noodle House,Chinese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Thai Restaurant,Italian Restaurant,Bank,Rental Car Location,Women's Store
17,North York,0.0,Golf Course,Dog Run,Pool,Mediterranean Restaurant,Deli / Bodega,Eastern European Restaurant,Drugstore,Discount Store,Diner,Dessert Shop


#### Cluster 2

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1.0,Playground,Cosmetics Shop,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
14,Scarborough,1.0,Playground,Park,Women's Store,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
23,North York,1.0,Park,Convenience Store,Bar,Bank,Women's Store,Dessert Shop,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run
25,North York,1.0,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
30,North York,1.0,Airport,Park,Playground,Women's Store,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
31,North York,1.0,Grocery Store,Park,Bank,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
40,East York,1.0,Park,Pizza Place,Coffee Shop,Convenience Store,Women's Store,Dance Studio,Drugstore,Dog Run,Discount Store,Diner
44,Central Toronto,1.0,Photography Studio,Swim School,Bus Line,Park,Comic Shop,Concert Hall,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
48,Central Toronto,1.0,Tennis Court,Park,Playground,Summer Camp,Curling Ice,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
50,Downtown Toronto,1.0,Park,Playground,Trail,Building,Women's Store,Dance Studio,Drugstore,Dog Run,Discount Store,Diner


#### Cluster 3

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,2.0,Baseball Field,Pool,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
97,North York,2.0,Baseball Field,Women's Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop


#### Cluster 4

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Print Shop,Women's Store,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
3,Scarborough,3.0,Coffee Shop,Korean Restaurant,Women's Store,Department Store,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
6,Scarborough,3.0,Department Store,Coffee Shop,Chinese Restaurant,Hobby Shop,Discount Store,Bus Station,Women's Store,Electronics Store,Eastern European Restaurant,Drugstore
15,Scarborough,3.0,Chinese Restaurant,Fast Food Restaurant,Pharmacy,Grocery Store,Coffee Shop,Pizza Place,Sandwich Place,Japanese Restaurant,Deli / Bodega,Dog Run
24,North York,3.0,Grocery Store,Discount Store,Pizza Place,Coffee Shop,Pharmacy,Gastropub,Drugstore,General Entertainment,Diner,Dessert Shop
28,North York,3.0,Coffee Shop,Restaurant,Fast Food Restaurant,Bridal Shop,Bank,Diner,Sushi Restaurant,Deli / Bodega,Middle Eastern Restaurant,Drugstore
29,North York,3.0,Massage Studio,Coffee Shop,Falafel Restaurant,Furniture / Home Store,Bar,Miscellaneous Shop,Women's Store,Department Store,Drugstore,Dog Run
34,North York,3.0,Coffee Shop,Pizza Place,Portuguese Restaurant,French Restaurant,Hockey Arena,Intersection,Dance Studio,Drugstore,Dog Run,Discount Store
38,East York,3.0,Gym,Fish & Chips Shop,Pet Store,Restaurant,Liquor Store,Bike Shop,Sporting Goods Shop,Sports Bar,Supermarket,Coffee Shop
46,Central Toronto,3.0,Coffee Shop,Spa,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Chinese Restaurant,Salon / Barbershop,Yoga Studio,Arts & Crafts Store


#### Cluster 5

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Bar,Women's Store,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dessert Shop
